In [1]:
# Colab bootstrap: clone & install your package in editable mode
import sys, os, subprocess, pathlib, textwrap

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    REPO = "https://github.com/acb-code/boilers.git"
    WORKDIR = "/content/boilers"
    if not os.path.exists(WORKDIR):
        subprocess.run(["git", "clone", REPO, WORKDIR], check=True)
    os.chdir(WORKDIR)
    subprocess.run([sys.executable, "-m", "pip", "install", "-U", "pip", "setuptools", "wheel"], check=True)
    subprocess.run([sys.executable, "-m", "pip", "install", '-e', '.[dev]'], check=True)

import torch
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())


Torch: 2.8.0+cu128 | CUDA: False


In [2]:
%load_ext autoreload
%autoreload 2

import torch, random, numpy as np
def set_seed(s=1337):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed()

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cpu


In [3]:
from boilers.data.datasets import moons
from boilers.models.logistic import LogisticClassifier
from boilers.models.mlp import MLP
from boilers.training.loop import fit, TrainCfg
import torch.nn as nn

train_dl, val_dl, meta = moons(batch_size=128)

# Try logistic regression
logreg = LogisticClassifier(meta["input_dim"], meta["num_classes"])
cfg = TrainCfg(epochs=5, lr=1e-2, optimizer="sgd")
hist = fit(logreg, train_dl, val_dl, cfg, loss_fn=nn.CrossEntropyLoss())

# Try a small MLP
mlp = MLP(input_dim=meta["input_dim"], hidden_sizes=(64,64), num_classes=meta["num_classes"])
cfg = TrainCfg(epochs=10, lr=1e-3, optimizer="adam")
hist = fit(mlp, train_dl, val_dl, cfg, loss_fn=nn.CrossEntropyLoss())


epoch 01 | train 0.4508 | val 0.3779 | acc 0.818
epoch 02 | train 0.3377 | val 0.3260 | acc 0.850
epoch 03 | train 0.3084 | val 0.3098 | acc 0.859
epoch 04 | train 0.2976 | val 0.3029 | acc 0.859
epoch 05 | train 0.2926 | val 0.2993 | acc 0.866
epoch 01 | train 0.5205 | val 0.3569 | acc 0.865
epoch 02 | train 0.3000 | val 0.2745 | acc 0.891
epoch 03 | train 0.2514 | val 0.2403 | acc 0.916
epoch 04 | train 0.2171 | val 0.2065 | acc 0.931
epoch 05 | train 0.1799 | val 0.1725 | acc 0.940
epoch 06 | train 0.1466 | val 0.1447 | acc 0.955
epoch 07 | train 0.1185 | val 0.1258 | acc 0.965
epoch 08 | train 0.1015 | val 0.1099 | acc 0.974
epoch 09 | train 0.0904 | val 0.1048 | acc 0.973
epoch 10 | train 0.0836 | val 0.1039 | acc 0.974
